In [6]:
import numpy as np
import random
import math
from IPython.display import display, clear_output

random.seed(0)

# Layer represents a MLP Layer
# It has two main properties:
#      - a weigth matrix containing the weights of the layer's neurons. Each line represents a neuron and 
#        the columns represent its corresponding weights
#      - a bias vector, containing the neurons's bias
# Since during the backpropagation we need to compute the weights variation using the old ones, the 
# updated_weights and updated_bias properties store the new values until the update method is called
class Layer:
    # Create a new Layer with 'size' neurons, each one linked to 'inputs_size' inputs
    def __init__(self, size, inputs_size):
        self.size = size
        self.inputs_size = inputs_size
        self.weights = np.array([[random.uniform(-0.1, 0.1) for j in range(inputs_size)] for i in range(size)])
        self.bias = np.array([random.uniform(-0.1,0.1) for i in range(size)])
        
        self.d_weights_current = np.zeros((size, inputs_size))
        self.d_bias_current = np.zeros(size)
        self.d_weights_old = np.zeros((size, inputs_size))
        self.d_bias_old = np.zeros(size)
    
    # update updates the weights and bias matrices with the values stored in the updated ones
    def update(self, eta, alpha):
        #self.weights = np.copy(self.updated_weights)
        #self.bias = np.copy(self.updated_bias)
        
        self.weights = self.weights + eta*self.d_weights_current + alpha*self.d_weights_old 
        self.bias = self.bias + eta*self.d_bias_current + alpha*self.d_bias_old
        
        self.d_weights_old = self.d_weights_current
        self.d_bias_old = self.d_bias_current
        
    # description prints a layer description
    def description(self):
        print("Layer Info")
        print("Weights: \n", self.weights)
        print("Bias: \n ", self.bias)

def logistic(x):
    return 1.0/(1.0+ math.exp(-x))

logistic_vec = np.vectorize(logistic)

def logistic_derivate(x):
    return x*(1.0-x)

In [202]:
class MLP:
    # MLP creation. One might pass the MLP layers as parameters or add them later using the add_layer method.
    # The classification parameter defines if the MLP will be used for a classification or regression problem
    def __init__(self, *layers, classifier=True):
        self.classifier = classifier
        if classifier:
            # Map each class label to a vector with a single 1
            # Ex: Class 0 -> [1,0]
            #     Class 1 -> [0,1]
            self.class_mapping = dict()  
            # Unmap each class vector to the corresponding class label
            # Ex: [1,0] -> Class 0 
            #     [0,1] -> Class 1
            self.class_unmapping = dict()
            
        self.layers = list()
        for layer in layers:
            self.add_layer(layer)
    
    # Shortcut to create a classifier MLP
    @classmethod
    def MLPClassifier(cls, *layers):
        return cls(classifier=True, *layers)   
    
    # Shortcut to create a regressor MLP
    @classmethod
    def MLPRegressor(cls, *layers):
        return cls(classifier=False, *layers)
    
    # add_layer adds a new layer on the MLP. It verifies whether or not the new layer is compatible with the MLP
    def add_layer(self, layer):
        # If there's already a layer in the MLP, verify if the new layer is compatible
        if len(self.layers) > 0:
            if layer.inputs_size != self.layers[-1].size:
                print("The new layer is incompatible with the MLP")
                print("Please, use a layer where each neuron has the same amount of inputs as the number" \
                     "of neurons in the MLP last layer")
        
        self.layers.append(layer)
    
    # description prints the info about the MLP layers
    def description(self):
        print("MLP Classifier?: ", self.classifier)
        print("-------------------------")
        print("MLP Info:")
        for layer, i in zip(self.layers, range(len(self.layers))):
            print("--- Layer: %d ---" % i)
            layer.description()
            
    # __get_class_mapping gets the class labels in the classes list and builds the mapping dicionaries
    # class_mapping and class_unmapping
    def __get_class_mapping(self, classes):
        class_labels = np.unique(classes)
        
        for c in range(len(class_labels)):
            class_label = class_labels[c]
            class_vector = np.zeros(len(class_labels))
            class_vector[c] = 1
    
            self.class_mapping[class_label] = class_vector
            
            # We can't use a list as a hash key. So transform it into a tuple
            self.class_unmapping[tuple(class_vector)] = class_label
        
    # __convert_class_labels_to_vectors converts a list with class labels to a list with 
    # vectors that maps each class label
    def __convert_class_labels_to_vectors(self, class_labels):
        return [self.class_mapping[c] for c in class_labels]
    
    # __convert_class_vectors_to_labels converts a list with class vectors to a list with 
    # the corresponding class labels
    def __convert_class_vectors_to_labels(self, class_vectors):
        return [self.class_unmapping[tuple(class_vector)] for class_vector in class_vectors]
        
        
    # fast_forward computes the ouput for a given input vector
    def fast_forward(self,input_v):
        # We need to store each layer input in order to perform the backpropagation
        self.inputs = list()
    
        # The input is applied in a layer weights matrix and the bias is added in the result
        # Then, the logistic function is applied to each layer neuron result
        # For a layer, we have a final output vector where each component i represents the output
        # of the neuron i
        for l in range(len(self.layers)-1):
            layer = self.layers[l]
            self.inputs.append(input_v)
            output = logistic_vec(layer.weights @ input_v + layer.bias)
            
            # The output of the current layer is the input of the next one
            input_v = output
            
        # Last layer
        self.inputs.append(input_v)
        layer = self.layers[-1]
        output = layer.weights @ input_v + layer.bias
        
        # If it's a classification problem, apply the softmax function on the output of the last layer
        # It it's a regression problem, the activation function of the last layer is the identity function
        if self.classifier:
            output = np.exp(output)/sum(np.exp(output))
          
        return output
    
    # train trains the MLP using the examples passed in the samples parameter
    # The expected output for each example must be passed in the classes parameter;
    # eta represents the MLP learning rate;
    # tol represents the error tolerance. The MLP is trained until the cumulative squared error for all example
    #     is less than the tol value
    # print_status prints the output for each example during the training phase
    def train(self, samples, classes, eta=0.5, alpha=0, tol=1e-2, epoch_max=2000, print_status=False):
        # Map the class labels to output vectors
        self.__get_class_mapping(classes)
        classes = self.__convert_class_labels_to_vectors(classes)
        
        print(self.class_mapping)
        print(self.class_unmapping)
                
        error = tol
        new_error = 3*tol
        epoch = 0
        
        # The training stops when the max number of epochs is reached or the Kramer and Sangiovanni-Vicentelly
        # criteria is valid. According to it, we can consider that the BP converged when the average mean squared
        # error is less than a given tolerance
        while (abs(new_error - error) > tol and epoch < epoch_max):
            epoch += 1
            error = 0
            new_error = 0
            
            for input_v, t in zip(samples, classes):  
                # ---- Compute the output for the given input vector ----
                output = self.fast_forward(input_v)
                
                # Compute the mean squared error before the backpropagation
                error_sample = pow((np.array(t)-np.array(output)),2)
                # We need to sum the error of each component when the output is a vector
                error += sum(error_sample)/len(samples)
                
                if (print_status == True):
                    print("\ttraining example: %s from class %s" % (input_v, t), end = " ")
                    print("y = ", output)

                    
                # ---- Backpropagation ----
                # Compute the new weights of each layer
                # Remark: the udpated weights are stored as a layer property and the layer is updated once 
                # the backpropagation is finished
                # It's necessary to do so in order to compute the delta value for the inner layers. We need 
                # to use the weights that caused the error to compute the delta instead of the updated weights
                for l in reversed(range(len(self.layers))): # Traverse the layers in reversed order
                    layer = self.layers[l]
             
                    deltas = list()
                    # Compute the delta for each layer neuron n
                    for n in range(len(layer.weights)):
                        # Last Layer
                        if l == (len(self.layers)-1):
                            delta = (t[n]-output[n])*logistic_derivate(output[n])
                            
                        # Inner Layer
                        else:
                            # output of the current layer is the input of the next one
                            neuron_output = self.inputs[l+1][n]
                            # weights of each neuron output
                            errors_weights = self.layers[l+1].weights[:,n]
                            
                            delta = np.dot(delta_next_layer,errors_weights)*logistic_derivate(neuron_output)
                              
                        # Computes the weights and bias variation for the neuron n
                        for w in range(len(layer.weights[n])):
                            layer.d_weights_current[n][w] = delta*self.inputs[l][w]
                        layer.d_bias_current[n] = delta*1 # bias input = 1
                        
                        #for w in range(len(layer.weights[n])):
                        #    layer.updated_weights[n][w] = layer.weights[n][w] + eta*delta*self.inputs[l][w]
                        #layer.updated_bias[n] = layer.bias[n] + (eta*delta*1) # bias input = 1

                        # Store the neuron delta
                        deltas.append(delta)
                    
                    # The neurons' delta of the current layer will be used to compute the deltas of the 
                    # next inner layer
                    delta_next_layer = np.array(deltas)
                     
                # Once the backpropagation is finished for the current example, update all the weigths and bias
                for layer in self.layers:
                    layer.update(eta, alpha)
                    
                # Compute the new error mean squared error
                output = self.fast_forward(input_v)
                error_sample = pow((np.array(t)-np.array(output)),2)
                new_error += sum(error_sample)/len(samples)
            
            # End of a epoch
            if epoch%1 == 0: # Print status only after each 100 iterations 
                clear_output(wait=True)
                display("End of epoch " + str(epoch) + ". Total Error = " + str(new_error))
        
        # End of training         
        clear_output(wait=True)
        display("End of epoch " + str(epoch) + ". Total Error = " + str(new_error))
        
    # predicts gets a list of input samples and returns a list with the predicted outputs
    def predict(self, samples):
        outputs = list()
        for input_v in samples:
            probs = self.fast_forward(input_v)
            class_pos = np.argmax(probs)
            output = np.zeros(len(probs))
            output[class_pos] = 1
            
            #outputs.append(self.class_unmapping[tuple(output)])
            outputs.append(output)
    
        return self.__convert_class_vectors_to_labels(outputs)

In [203]:
random.seed(0)

samples = [[0,0], [1,0], [1,1], [0,1]]
classes = [[1,0],[0,1],[1,0],[0,1]]

mlp = MLP.MLPClassifier(Layer(5,2), Layer(2,5))
mlp.train([[0,0], [1,0], [1,1], [0,1]], [0,1,0,1], eta=0.5, alpha=0.8, tol=1e-4, print_status=False)
predicted = mlp.predict(samples)

'End of epoch 966. Total Error = 0.003344547389935799'

In [204]:
mlp.predict([[0,0],[0,1],[1,0],[1,1]])

[0, 1, 1, 0]

In [209]:
import pandas as pd

df = pd.read_csv('winequality-red.csv')
df.head(5)

,Unnamed: 0,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,category
0,0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,Mid
1,1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,Mid
2,2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,Mid
3,3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,Mid
4,4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,Mid


In [210]:
# Separação dos Dados
inputs = df[df.columns[1:-1]].values
classes = df[df.columns[-1]].values

print(inputs[0:2,:])
print(classes)

[[ 7.4     0.7     0.      1.9     0.076  11.     34.      0.9978  3.51
   0.56    9.4   ]
 [ 7.8     0.88    0.      2.6     0.098  25.     67.      0.9968  3.2
   0.68    9.8   ]]
['Mid' 'Mid' 'Mid' ... 'Mid' 'Mid' 'Mid']


In [184]:
np.unique(classes)

array(['Bad', 'Good', 'Mid'], dtype=object)

In [185]:
n_classes = len(np.unique(classes))
n_classes


map_classes = {"Bad": [1,0,0], 
               "Mid": [0,1,0], 
               "Good": [0,0,1]}

mapped_classes = [map_classes[c] for c in classes]

#mapped_classes

In [ ]:
N = len(inputs[0])
mlp = MLP(Layer(2*N, N), Layer(n_classes, 2*N))

mlp.train(inputs, mapped_classes, eta=0.5, alpha=1.2, tol=1e-2, print_status=False)


CUIDADO: Problema. math range na função math.exp -> saturação da saída dos neurônios provavelmente devido ao fato dos atributos não estarem normalizados entre 0 e 1.

Outro problema: Os atributos não possuem mesma variância.

Pré-Processamento: Colocar mais exemplos para que todas classes fiquem com mesma quantidade de exemplos

In [ ]:
unique, counts = np.unique(classes, return_counts=True)

In [ ]:
unique

In [ ]:
counts

In [ ]:
b = np.where(classes == 'Good')

In [186]:
def data_normalization(data):
    normalized_columns = list()
    for c in range(len(data[0])):
        col = inputs[:,c]
        normalized_columns.append((col - np.mean(col))/np.std(col))

    return np.array(normalized_columns).T

normalized = data_normalization(inputs)
normalized

array([[-0.52835961,  0.96187667, -1.39147228, ...,  1.28864292,
        -0.57920652, -0.96024611],
       [-0.29854743,  1.96744245, -1.39147228, ..., -0.7199333 ,
         0.1289504 , -0.58477711],
       [-0.29854743,  1.29706527, -1.18607043, ..., -0.33117661,
        -0.04808883, -0.58477711],
       ...,
       [-1.1603431 , -0.09955388, -0.72391627, ...,  0.70550789,
         0.54204194,  0.54162988],
       [-1.39015528,  0.65462046, -0.77526673, ...,  1.6773996 ,
         0.30598963, -0.20930812],
       [-1.33270223, -1.21684919,  1.02199944, ...,  0.51112954,
         0.01092425,  0.54162988]])

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
print(scaler.fit_transform(inputs))

In [ ]:
N = len(inputs[0])
mlp = MLP(Layer(N, N), Layer(n_classes, N))

mlp.train(normalized, mapped_classes, eta=0.5, alpha=0.5, tol=1e-4, epoch_max=500, print_status=False)

In [ ]:
predicted = mlp.predict(normalized)

In [ ]:
mapped_classes[8]

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(np.array(mapped_classes), np.array(predicted))

In [ ]:
from sklearn.metrics import confusion_matrix
unmap_classes = {
            (1,0,0):"Bad",
            (0,1,0):"Mid", 
            (0,0,1):"Good"}

unmapped_predicted = [unmap_classes[tuple(c)] for c in predicted]

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(classes, unmapped_predicted)

## Balanceado

In [211]:
unique, counts = np.unique(classes, return_counts=True)
print(unique)
print(counts)

['Bad' 'Good' 'Mid']
[  63  217 1319]


In [212]:
examples = min(counts)
examples

63

In [213]:
bad_indices = np.random.choice(np.where(classes == 'Bad')[0], examples)
good_indices = np.random.choice(np.where(classes == 'Good')[0], examples)
mid_indices = np.random.choice(np.where(classes == 'Mid')[0], examples)

under_sampled_examples = normalized[bad_indices]
under_sampled_classes = classes[bad_indices]

under_sampled_examples = np.append(under_sampled_examples,normalized[good_indices], axis=0)
under_sampled_classes = np.append(under_sampled_classes, classes[good_indices], axis=0)

under_sampled_examples = np.append(under_sampled_examples,normalized[mid_indices], axis=0)
under_sampled_classes = np.append(under_sampled_classes, classes[mid_indices], axis=0)

len(under_sampled_examples)
print(len(under_sampled_classes))

189


In [214]:
n_classes = len(np.unique(classes))
print("n classes:", n_classes)

n classes: 3


In [215]:
N = len(inputs[0])
mlp = MLP(Layer(N, N), Layer(n_classes, N))

mlp.train(under_sampled_examples, under_sampled_classes, eta=0.5, alpha=0.5, tol=1e-8, epoch_max=500, print_status=False)

'End of epoch 500. Total Error = 0.0007465955891267764'

In [216]:
predicted = mlp.predict(under_sampled_examples)

In [221]:
from sklearn.metrics import accuracy_score
accuracy_score(under_sampled_classes, predicted)

1.0

In [222]:
confusion_matrix(under_sampled_classes, predicted)

array([[63,  0,  0],
       [ 0, 63,  0],
       [ 0,  0, 63]])

### Com SKLearn

In [ ]:
clf = MLPClassifier(solver='sgd', alpha=1e-5, activation='logistic', learning_rate='constant', tol=1e-10,
                    learning_rate_init=0.5, momentum=0.5, hidden_layer_sizes=(N), random_state=1,
                    shuffle=False, max_iter=6000)


clf.fit(under_sampled_examples, mapped_under_sampled_classes)

In [ ]:
predicted = clf.predict(under_sampled_examples)
accuracy_score(np.array(mapped_under_sampled_classes), predicted)

## MLP Sklearn

In [ ]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(solver='sgd', alpha=1e-5, activation='logistic', learning_rate='constant', tol=1e-10,
                    learning_rate_init=0.5, momentum=0.5, hidden_layer_sizes=(N), random_state=1,
                    shuffle=False, max_iter=1000)


clf.fit(normalized, mapped_classes)

In [ ]:
clf.loss_

In [ ]:
mapped_classes[8]

In [ ]:
predicted = clf.predict(normalized)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(np.array(mapped_classes), predicted)